In [52]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score , precision_score
import os, sys, gc, warnings, random, datetime, math

In [53]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [54]:
SEED = 42
seed_everything(SEED)

In [55]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
gyeong =pd.concat([train, val])
data = pd.concat([train, val, test])

In [56]:
data = data.reset_index()
data.drop('index',axis= 1, inplace = True)

In [57]:
gyeong = gyeong.reset_index()
gyeong.drop('index',axis= 1, inplace = True)

In [58]:
TARGET = 'fr_yn'
LOCAL_TEST = False

In [59]:
lgb_params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'seed' :SEED,
                    'early_stopping_rounds':100,
                    #'metric' : 'binary_logloss'
                    'metric' : 'auc'
                
                }

In [60]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1_score', f1_score(y_true, y_hat), True

In [61]:
rm_cols = ['id']

In [62]:
features_columns = [col for col in list(data) if col not in rm_cols]

In [63]:
train = data[:(len(train)+len(val))]
test = data[-len(test):]
dataset = [train, test]

for df in dataset:
    categorical_cols1 = df.select_dtypes(['object']).columns
    for col in categorical_cols1:
        df[col] = pd.Categorical(df[col]).codes

In [64]:
# def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=5):
#     folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

#     X,y = tr_df[features_columns], tr_df[target]    
#     P,P_y = tt_df[features_columns], tt_df[target]  

#     tt_df = tt_df[['id',target]]    
#     predictions = np.zeros(len(tt_df))
#     oof = np.zeros(len(tr_df))
    
#     for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
#         print('Fold:',fold_)
#         tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
#         vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
#         print(len(tr_x),len(vl_x))
#         tr_data = lgb.Dataset(tr_x, label=tr_y)
        
#         if LOCAL_TEST:
#             vl_data = lgb.Dataset(P, label=P_y) 
#         else:
#             vl_data = lgb.Dataset(vl_x, label=vl_y)  

#         estimator = lgb.train(
#             lgb_params,
#             tr_data,
#             #feval=lgb_f1_score,
#             valid_sets = [tr_data, vl_data],
#             verbose_eval = 200,
#         )   
        
#         pp_p = estimator.predict(vl_x)
#         print('ddddd')
#         print(pp_p)
#         print('ddddd')
#         predictions += pp_p/NFOLDS
#         oof_preds = estimator.predict(P)
#         oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())
        

#         if LOCAL_TEST:
#             feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
#             print(feature_imp)
        
#         del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
#         gc.collect()
        
#     tt_df['prediction'] = predictions
#     print('OOF f1score:', f1_score(y, oof))
    
#     return tt_df

In [77]:
########################### Model
import lightgbm as lgb

def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
    tt_df[target] = 0
    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    tt_df = tt_df[['id',target]]    
    predictions = np.zeros(len(tt_df))
    #print('X롸 y확인해보가')
    #print(X)
    #print('X끝')
    #print(y)
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
        print('trytrytyr')
        #print(trn_idx)
        #print(val_idx)
        #print(vl_x)
        #print('valalalal')
        #print(vl_y)
        
        #print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        #print('tr_x다')
        #print(tr_y)
       # if LOCAL_TEST:
        #    vl_data = lgb.Dataset(P, label=P_y) 
        #else:
        vl_data = lgb.Dataset(vl_x, label=vl_y)  
        #print("vl_x다")
        #print(vl_y)
        estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(P)
        print(pp_p)
        predictions += pp_p/NFOLDS

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    
    return tt_df
## -------------------

In [78]:
make_predictions (train,test, features_columns, TARGET, lgb_params,NFOLDS=2)

Fold: 0
trytrytyr
[    2     5     8 ... 66093 66095 66096]
[    0     1     3 ... 66091 66092 66094]
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1
[0.13429466 0.13429466 0.13429466 ... 0.13429466 0.13429466 0.13429466]
Fold: 1
trytrytyr
[    0     1     3 ... 66091 66092 66094]
[    2     5     8 ... 66093 66095 66096]
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1
[0.13318224 0.13318224 0.13318224 ... 0.13318224 0.13318224 0.13318224]


,id,fr_yn,prediction
66097,116766,0,0.133738
66098,120945,0,0.133738
66099,105476,0,0.133738
66100,105148,0,0.133738
66101,98350,0,0.133738
66102,97406,0,0.133738
66103,112881,0,0.133738
66104,78230,0,0.133738
66105,125083,0,0.133738
66106,77050,0,0.133738
